### Generating a toy-case topology

In [2]:
import numpy as np 

In [3]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 20                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [3]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [4]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [5]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [6]:
np.trace(X.T @ L_f @ X)

10690.130358964443

In [7]:
np.trace(Y.T @ L_f @ Y)

120.33265407669947

____________

In [14]:
from itertools import combinations
from tqdm import tqdm

In [8]:
def premultiplier(Xu, Xv):
    uu = np.linalg.inv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = np.linalg.inv(Xv @ Xv.T)
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return ((uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0])) @ vu - np.eye(uu.shape[0])) @ uu

def chi_v(uu, uv, vv, vu):

    return (uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0]))

In [12]:
T = 0

maps = {
    edge : {
        edge[0] : np.zeros((d,d)),
        edge[1] : np.zeros((d,d))
    }
for edge in combinations(nodes, 2)
}

In [24]:
for e in tqdm(combinations(nodes,2)):
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]
    uu, uv, vv, vu = premultiplier(X_u, X_v)

    maps[e][u] = chi_u(uu, uv, vv, vu)
    maps[e][v] = chi_u(uu, uv, vv, vu)
    
    T += np.trace(maps[e][u]) + np.trace(maps[e][u])

21it [00:00, 2623.86it/s]


In [25]:
def reconstructed_laplacian_metrics(
        V:int,
        edges:list,
        d:int,
        maps:dict,
        X:np.array,
        L_f:np.array,
        ) -> dict:
    
    '''
    Retrieve the first E edges based on their expressed energy and compute metrics of similarities with the original laplacian.

    Parameters:
    - V (int): number of nodes.
    - edges (list): edges of the underlying graph
    - d (int): stalks dimensions
    - maps (dict) dictionary of restriction maps 
    - X (np.array): dataset of shape (V*d, N) of smooth signals
    - L_f (np.array): groundthruth for the sheaf laplacian
    
    Returns:
    - dict: dictionary containing three metrics as chosen by Hansen: average entrywise L2 and L1 reconstruction error, precision in recovering the graph underlying the sheaf
    '''

    E = len(edges)

    all_edges = list(combinations(range(V), 2))

    energies = {
        e : 0
        for e in all_edges
        }
    
    for e in all_edges:
        u = e[0]
        v = e[1]

        X_u = X[u*d:(u+1)*d,:]
        X_v = X[v*d:(v+1)*d,:]


        F_u = maps[e][u]
        F_v = maps[e][u]

        
        energies[e] = np.linalg.norm(F_u @ X_u - F_v @ X_v)

    retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

    B_hat = np.zeros((d*E, d*V))

    for i in range(E):
        edge = retrieved[i][0]

        u = edge[0] 
        v = edge[1] 

        B_u = maps[edge][u]
        B_v = maps[edge][v]

        B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
        B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

    L_f_hat = B_hat.T @ B_hat

    return {
        "AvgEntryWiseED_L2" : np.sqrt(np.sum((L_f - L_f_hat)**2)) / L_f.size,
        "AvgEntryWiseED_L1" : np.sqrt(np.sum(np.abs(L_f - L_f_hat))) / L_f.size,
        "SparsityAccuracy" : len(set(list(map(lambda x: x[0], retrieved))).intersection(set(edges))) / E
        }


In [26]:
for N in range(1,100):
    maps_ = {
        edge : {
            edge[0] : N/T * maps[edge][edge[0]],
            edge[1] : N/T * maps[edge][edge[1]]
        }
        for edge in combinations(nodes, 2)
    }
    print(N,reconstructed_laplacian_metrics(len(nodes), edges, d, maps_, Y, L_f))

1 {'AvgEntryWiseED_L2': 0.08243484378858681, 'AvgEntryWiseED_L1': 0.042754948419883934, 'SparsityAccuracy': 0.25}
2 {'AvgEntryWiseED_L2': 0.08243395792320231, 'AvgEntryWiseED_L1': 0.04275495786216986, 'SparsityAccuracy': 0.25}
3 {'AvgEntryWiseED_L2': 0.08243248155811667, 'AvgEntryWiseED_L1': 0.0427549845376903, 'SparsityAccuracy': 0.25}
4 {'AvgEntryWiseED_L2': 0.08243041480917125, 'AvgEntryWiseED_L1': 0.04275502285511217, 'SparsityAccuracy': 0.25}
5 {'AvgEntryWiseED_L2': 0.08242775783856159, 'AvgEntryWiseED_L1': 0.04275507212031839, 'SparsityAccuracy': 0.25}
6 {'AvgEntryWiseED_L2': 0.08242451085485455, 'AvgEntryWiseED_L1': 0.042755132333271134, 'SparsityAccuracy': 0.25}
7 {'AvgEntryWiseED_L2': 0.08242067411301086, 'AvgEntryWiseED_L1': 0.04275520349392413, 'SparsityAccuracy': 0.25}
8 {'AvgEntryWiseED_L2': 0.08241624791441247, 'AvgEntryWiseED_L1': 0.04275528560222273, 'SparsityAccuracy': 0.25}
9 {'AvgEntryWiseED_L2': 0.08241123260689494, 'AvgEntryWiseED_L1': 0.04275537865810385, 'Sparsit